In [6]:
import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from collections import defaultdict,Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import string
nltk.download('stopwords')
stop=set(stopwords.words('english'))
plt.style.use('seaborn')

from plotly import tools
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import textstat
from tqdm import tqdm
from statistics import *
import plotly.express as px
from plotly.subplots import make_subplots

import plotly.graph_objects as go

ModuleNotFoundError: No module named 'nltk'

In [5]:
!pip install nltk

In [2]:

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
#from kaggle_datasets import KaggleDatasets
#import transformers
#from tqdm.notebook import tqdm
#from tokenizers import BertWordPieceTokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import concatenate
#from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.layers import CuDNNLSTM
import gc
import os

Using TensorFlow backend.


In [11]:
import keras
print(keras.__version__)

2.2.4


Using TensorFlow backend.


In [12]:
conda install keras==2.3.0

SyntaxError: invalid syntax (<ipython-input-12-c6cfd5dfbc17>, line 1)